## Problem Statement : Hospital Patient Data Analysis

### Context:
A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.


In [64]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [65]:
patient_df = pd.read_csv('Patient_Data.csv')
patient_df

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45
5,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00


#### 1.	Load the patient dataset and show summary with info().

In [66]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


#### 2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].

In [67]:
billing_cols = ['PatientID', 'Department', 'Doctor', 'BillAmount']
df_billing_view = patient_df[billing_cols]
df_billing_view

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN
5,101,Cardiology,Dr. Smith,5000.0


#### 3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].

In [68]:
drop_cols = patient_df.drop(columns=['ReceptionistID', 'CheckInTime'], inplace=True)
drop_cols

#### 4.	Use groupby to find total bill amount per department.

In [69]:
department_bill = patient_df.groupby('Department')['BillAmount'].sum()
department_bill

Department
Cardiology     16200.0
Dermatology        0.0
Neurology          0.0
Orthopedics     7500.0
Name: BillAmount, dtype: float64

#### 5.	Remove duplicate patient records based on PatientID.

In [70]:
patient_df.duplicated().sum()

np.int64(1)

In [71]:
df_no_duplicates = df_cleaned.drop_duplicates(subset='PatientID')
df_no_duplicates

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


#### 6.	Fill missing BillAmount values with the mean bill amount.

In [72]:
mean_bill = patient_df['BillAmount'].mean()
patient_df['BillAmount'] = patient_df['BillAmount'].fillna(mean_bill, inplace=True)
patient_df

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,None
1,102,Bob,Neurology,Dr. John,None
2,103,Charlie,Orthopedics,Dr. Lee,None
3,104,David,Cardiology,Dr. Smith,None
4,105,Eva,Dermatology,Dr. Rose,None
5,101,Alice,Cardiology,Dr. Smith,None


#### 7.	Merge the billing dataset with patient dataset on PatientID.

In [73]:
billing_df = pd.read_csv('Billing_Data.csv')
billing_df

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [74]:
billing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


In [75]:
df_merged = pd.merge(patient_df, billing_df, on='PatientID', how='left')
df_merged

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,None,2000,3000
1,102,Bob,Neurology,Dr. John,None,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,None,2500,5000
3,104,David,Cardiology,Dr. Smith,None,3000,3200
4,105,Eva,Dermatology,Dr. Rose,None,1000,4000
5,101,Alice,Cardiology,Dr. Smith,None,2000,3000


#### 8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).

In [76]:
new_data={
    'PatientID': [106, 107],
    'Name': ['Frank', 'Grace'],
    'Department': ['Dermatology', 'Cardiology'],
    'Doctor': ['Dr. Brown', 'Dr. Smith'],
    'BillAmount': [7200.0, 6400.0]
}

In [77]:
new_patients = pd.DataFrame(new_data)
new_patients

,PatientID,Name,Department,Doctor,BillAmount
0,106,Frank,Dermatology,Dr. Brown,7200.0
1,107,Grace,Cardiology,Dr. Smith,6400.0


In [78]:
df_all_patients = pd.concat([df_no_duplicates, new_patients], ignore_index=True)
df_all_patients

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN
5,106,Frank,Dermatology,Dr. Brown,7200.0
6,107,Grace,Cardiology,Dr. Smith,6400.0


#### 9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).

In [79]:
billing_data = {
    'InsuranceCovered': [3900, 5500, 6400, 5200, 6700, 5600, 4600],
    'FinalAmount': [4500, 7000, 5000, 7200, 6500, 7200, 6400]
}

#### •	Final cleaned dataset with accurate billing info.

In [80]:
new_billing_cols = pd.DataFrame(billing_data)
new_billing_cols

,InsuranceCovered,FinalAmount
0,3900,4500
1,5500,7000
2,6400,5000
3,5200,7200
4,6700,6500
5,5600,7200
6,4600,6400


#### •	All missing values handled, merged dataset across PatientID.

In [81]:
df_final = pd.concat([df_all_patients, new_billing_cols], axis=1)
df_final

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,3900,4500
1,102,Bob,Neurology,Dr. John,NaN,5500,7000
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,6400,5000
3,104,David,Cardiology,Dr. Smith,6200.0,5200,7200
4,105,Eva,Dermatology,Dr. Rose,NaN,6700,6500
5,106,Frank,Dermatology,Dr. Brown,7200.0,5600,7200
6,107,Grace,Cardiology,Dr. Smith,6400.0,4600,6400


#### •	Ability to perform further analytics on department-wise revenue or doctor performance.

In [82]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PatientID         7 non-null      int64  
 1   Name              7 non-null      object 
 2   Department        7 non-null      object 
 3   Doctor            7 non-null      object 
 4   BillAmount        5 non-null      float64
 5   InsuranceCovered  7 non-null      int64  
 6   FinalAmount       7 non-null      int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 524.0+ bytes


In [84]:
df_final.groupby('Department')['FinalAmount'].sum()

Department
Cardiology     18100
Dermatology    13700
Neurology       7000
Orthopedics     5000
Name: FinalAmount, dtype: int64

In [86]:
df_final.groupby('Doctor')['FinalAmount'].mean()

Doctor
Dr. Brown    7200.000000
Dr. John     7000.000000
Dr. Lee      5000.000000
Dr. Rose     6500.000000
Dr. Smith    6033.333333
Name: FinalAmount, dtype: float64